<a href="https://colab.research.google.com/github/BBrain778/CNN/blob/main/data_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#主題:使用CNN分類衛星影像中的災害從而加速災害應對




## 1.1 安裝套件(若在colab訓練每次都需要執行)

In [ ]:
!pip install fastbook -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 1.2 讀取套件

In [ ]:
from fastbook import *
from fastai.vision.widgets import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
import shutil
import fastai;
print('fastai verison:', fastai.__version__)
print('torch version:', torch.__version__)

fastai verison: 2.7.19
torch version: 2.6.0+cu124



## 1.3 準備資料集

In [ ]:
#將 Google Drive 掛載到 /content/drive，以便存取或保存檔案
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#set project name
project = 'demo'

path = Path('/content/drive/MyDrive/DeepLearning/0225/dataset/'+project)

#Key is the name of classes; value is the keyword that which search engine uses for searching
keywords = {'Satellite photos of the fire': 'Satellite photos of the fire', 'Drought satellite photos': 'Drought satellite photos', 'Satellite photos of floods': 'Satellite photos of floods', 'Satellite photos of volcanic eruptions': 'Satellite photos of volcanic eruptions', 'Satellite photos of heavy rain': 'Satellite photos of heavy rain', 'Typhoon satellite photos': 'Typhoon satellite photos'}
array = keywords.items()
#Satellite photos of the fire火災衛星照片
#Drought satellite photos乾旱衛星照片
#Satellite photos of floods水災衛星照片
#Satellite photos of volcanic eruptions火山爆發衛星照片
#Satellite photos of heavy rain豪雨衛星照片
#Typhoon satellite photos颱風衛星照片

project_folder = f'/content/drive/MyDrive/DeepLearning/0225/dataset/{project}'

In [ ]:
if not path.exists():
    !mkdir -p {project_folder}
else:
    print('The Project Folder exists; it will be removed and created again.')
    shutil.rmtree(project_folder)
    !mkdir -p {project_folder}

#網路爬蟲下載圖片
for key,value in array:
    print(key,value)
    dest = (path/key)
    dest.mkdir(exist_ok=True)
    urls = search_images_ddg(f' {value}',max_images=100)
    download_images(dest, urls=urls)

The Project Folder exists; it will be removed and created again.
Satellite photos of the fire Satellite photos of the fire
Drought satellite photos Drought satellite photos
Satellite photos of floods Satellite photos of floods
Satellite photos of volcanic eruptions Satellite photos of volcanic eruptions
Satellite photos of heavy rain Satellite photos of heavy rain
Typhoon satellite photos Typhoon satellite photos


## 1.4 清洗資料

In [ ]:
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink) #unlink broken images

(#31) [None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None...]

## 2.1 設定訓練資料路徑

In [ ]:
path = Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo')
path.ls()

(#6) [Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of the fire'),Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Drought satellite photos'),Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of floods'),Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of volcanic eruptions'),Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of heavy rain'),Path('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Typhoon satellite photos')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#建立模型權重儲存路徑
myPath='/content/drive/MyDrive/DeepLearning/0225/models'
!mkdir -p $myPath

## 2.2 資料讀取框架

In [ ]:
dataset = DataBlock(
    blocks=(ImageBlock, CategoryBlock), #定義輸入和輸出的資料類型(輸入:圖片/輸出:分類標籤)
    get_items=get_image_files,  #告訴 fastai 如何獲取資料
    splitter=RandomSplitter(valid_pct=0.2, seed=42),#20%的資料用於驗證集，剩下的80%用於訓練集
    item_tfms=Resize(224),#將所有圖片調整為 224x224 像素
    get_y=parent_label,#從圖片的父資料夾名稱獲取標籤
)

#利用框架正式讀取資料
# 批次大小（batch size），每次訓練或驗證時處理 16 張圖片
# 使用 16 個工作線程來並行載入資料，加快資料準備速度（特別在 GPU 環境中很有用）
dls = dataset.dataloaders(path,bs=16,num_workers=16) #dataset.dataloaders 是從你的數據集（dataset）生成一個數據加載器



#讀取結果
#print(dls.c, dls.vocab, len(dls.train_ds), len(dls.valid_ds)) #dls.c=類別數量(5) dls.vocab=標籤詞彙表 len(dls.train_ds)=訓練樣本數量(80%) len(dls.valid_ds)=驗證數量(20%)
#print(dls.c, dls.vocab, len(dls.train_ds), len(dls.valid_ds), len(dls.test_ds))

# 從驗證集分出測試集 (將 20% 分成 10% 驗證 + 10% 測試)
all_valid_items = dls.valid_ds.items  # 獲取驗證集的所有檔案
valid_test_splitter = RandomSplitter(valid_pct=0.5, seed=42)  # 從驗證集中再分一半
valid_idx, test_idx = valid_test_splitter(all_valid_items)

# 更新驗證集 (只保留 10%)
dls.valid_ds = dls.valid_ds.new_empty()
dls.valid_ds.items = [all_valid_items[i] for i in valid_idx]  # Update items in the new dataset

dls.valid = dls.valid.new(get_items=dls.valid_ds.items)
# 建立測試集 DataLoader
test_items = [all_valid_items[i] for i in test_idx]
test_dls = dls.valid.new(get_items=test_items)

# 讀取結果
print(f"類別數量: {dls.c}")
print(f"標籤詞彙表: {dls.vocab}")
print(f"訓練集樣本數: {len(dls.train_ds)} ({len(dls.train_ds)/len(get_image_files(path))*100:.1f}%)")
print(f"驗證集樣本數: {len(dls.valid_ds)} ({len(dls.valid_ds)/len(get_image_files(path))*100:.1f}%)")
print(f"測試集樣本數: {len(test_dls.dataset)} ({len(test_dls.dataset)/len(get_image_files(path))*100:.1f}%)")


AttributeError: property 'valid_ds' of 'DataLoaders' object has no setter

## 2.3 讀取圖檔結果

In [ ]:
print('訓練資料')
dls.show_batch(max_n=9, nrows=1) #從訓練集中抽出9個樣本   nrows=顯示的行數
#這邊顯示出來的圖片和標籤是我提供給fastai的資料，不是模型自己預測的結果


## 3.1選擇模型架構以及對應的預測訓練權重

NOTE: metrics是模型訓練人員觀察的指標，可設定多個

In [ ]:
# vision_learner是 fastai 庫中專門用於圖像視覺任務的函數，用來構建一個圖像分類模型
# dls 包含了訓練數據和驗證數據（通常是圖像數據及其標籤），並且已經被預處理，用來告訴模型“這些是我的數據，準備好用它們來訓練吧”
# resnet34 是模型的架構（architecture），具體來說是 ResNet-34，一種經典的卷積神經網絡（CNN），特別適合圖像分類任務
# metrics=[accuracy, error_rate]這些是在訓練過程中用來評估模型性能的指標
# pretrained=True表示使用預訓練的權重來初始化模型，ResNet-34 已經在 ImageNet（一個包含數百萬張圖像的大型數據集）上訓練過，因此它已經學會了識別圖像中的基本特徵（比如邊緣、形狀等）



learn = vision_learner(dls, resnet34, metrics=[accuracy, error_rate], pretrained=True)
learn.fit_one_cycle(3, 1e-3)

#總結
#建了一個基於 ResNet-34 的圖像分類模型，準備用 dls 中的數據進行訓練，並且希望看到準確率和錯誤率作為評估指標。模型從預訓練權重開始，而不是從隨機初始化的權重開始



## 3.2儲存第一次訓練好的權重

In [ ]:
myModel=myPath+'/resnet_stage-1.pkl'
learn.export(myModel)

## 3.3解凍權重再次訓練

In [ ]:
# 調整學習率（learning rate）的過程
learn.unfreeze()
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep), end_lr=5, num_it=200) #這行程式碼的作用是尋找一個最佳的學習率範圍，幫助你選擇一個能夠加快訓練並且穩定收斂的學習率

print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learn.fit_one_cycle(6, lr_max=2.75e-04)
#用 One Cycle Learning Rate 策略進行 6 次訓練，並在訓練過程中將學習率最大值設置為 2.75e-04

## 3.4儲存新的權重

In [ ]:
myModel=myPath+'/resnet_stage-2.pkl'
learn.export(myModel)

## 4.1結果檢核(Confusion Matrix)

In [ ]:
#contains interpretation methods for classification models
interp = ClassificationInterpretation.from_learner(learn)
#Plot the confusion matrix
interp.plot_confusion_matrix()

### 解釋一下confusion matrix
*   elephant (大象)：20/20，完全正確
*   giraffe (長頸鹿)：18/18，完全正確
*   hippo (河馬)：20/21，幾乎完全正確
*   zebra (斑馬)：19/19，完全正確
*   猴子 (monkey) 類別有誤分類
    *   14 張猴子圖片中，有 13 張正確分類，但 1 張被錯誤分類為河馬 (hippo)









In [ ]:
interp.plot_top_losses(5, nrows=5) #顯示前 5 個錯誤最嚴重的分類結果

## 預測

In [ ]:
## 模型位置
from fastbook import *
from fastai.vision.widgets import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

## 5.1讀取先前訓練好的權重

In [ ]:
myPath='/content/drive/MyDrive/DeepLearning/0225/models'
myModel=myPath+'/resnet_stage-2.pkl'
learn = load_learner(myModel)

## 5.2讀取檔案並送入模型預測

In [ ]:
##執行預測 -method I
# get_image_files is now available
fnames_fire = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of the fire')
fnames_Drought = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Drought satellite photos')
fnames_floods = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of floods')
fnames_volcanic = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of volcanic eruptions')
fnames_heavy_rain = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of heavy rain')
fnames_Typhoon = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Typhoon satellite photos')


print(f"Number of elephant images: {len(fnames_fire)}")
print(f"Number of elephant images: {len(fnames_Drought)}")
print(f"Number of elephant images: {len(fnames_floods)}")
print(f"Number of elephant images: {len(fnames_volcanic)}")
print(f"Number of elephant images: {len(fnames_heavy_rain)}")
print(f"Number of elephant images: {len(fnames_Typhoon)}")

In [ ]:
# Check if the list has enough elements before accessing them
if len(fnames_fire) > 3:
    pred_class,pred_idx,outputs = learn.predict(fnames_fire[3])
    print("Actual: fire, Predicted = {}".format(pred_class))
else:
    print("fnames_fire does not have enough elements to access index 3")


## ROC curve



*   橫軸是假陽性率（FPR），縱軸是真陽性率（TPR）
*   ROC 曲線（Receiver Operating Characteristic Curve，接收者操作特徵曲線）是一種用於評估分類模型性能的圖形工具，特別是在二元分類問題中（例如判斷「是/否」、「正/負」）
*   AUC（Area Under the Curve） 是 ROC 曲線下的面積，範圍從 0 到 1
*   AUC 的解釋
  *   AUC=1：完美模型，100%正確分類。
  *   AUC=0.5：隨機猜測，模型毫無區分能力（ROC 曲線是一條對角線）。
  *   AUC<0.5：模型比隨機猜測還差（可能是標籤錯誤或模型反向預測）。











In [ ]:
from sklearn.metrics import roc_curve, auc

preds, y, loss = learn.get_preds(with_loss=True)
preds, y = learn.get_preds(dl=dls.valid)
# get accuracy
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format( 100* round( float(acc), 4)))

class_num = dls.c
class_name = dls.vocab

plt.figure(figsize=(8, 8))
for i in range(class_num) :
  probs = np.array(preds[:, i])
  #compute ROC curve
  fpr, tpr, thresholds = roc_curve(y, probs, pos_label=i)
  #compute area under the curve
  roc_auc = auc(fpr, tpr)
  plt.plot(fpr, tpr, label='{0} (AUC={1})'.format( class_name[i], round(roc_auc, 3)))

plt.plot([0,1], [0,1], color='navy', linestyle='--')
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.axis('square')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve")
plt.legend(loc="lower right")

##6.V###tsisualization with Frad-CAM

Class Activation Mapping（類激活映射，簡稱CAM）是一種用於解釋卷積神經網路（CNN）預測的可視化技術。它可以幫助我們理解CNN在圖像分類任務中，哪些區域對最終預測貢獻最大。簡單來說，CAM生成一個熱圖（heatmap），顯示圖像中哪些部分對某個特定類別的預測最重要。



In [ ]:
class Hook():
  def __init__(self,m):
    self.hook = m.register_forward_hook(self.hook_fn)
  def hook_fn(self,m,i,o):self.stored = o.detach().clone()
  def __enter__(self, *args):return self
  def __exit__(self, *args):self.hook.remove()

In [ ]:
class HookBwd():
  def __init__(self,m):
    self.hook = m.register_backward_hook(self.hook_fn)
  def hook_fn(self,m,gi,go):self.stored = go[0].detach().clone()
  def __enter__(self, *args):return self
  def __exit__(self, *args):self.hook.remove()

##讀取要繪製的影像

In [ ]:
fnames_fire = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of the fire')
test_dl = learn.dls.test_dl(fnames_fire, with_labels=True)

fnames_Drought = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Drought satellite photos')
test_dl = learn.dls.test_dl(fnames_Drought, with_labels=True)

fnames_floods = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of floods')
test_dl = learn.dls.test_dl(fnames_floods, with_labels=True)

fnames_volcanic = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of volcanic eruptions')
test_dl = learn.dls.test_dl(fnames_volcanic, with_labels=True)

fnames_heavy_rain = get_image_files('/content/drive/MyDrive/DeepLearning/0225/dataset/demo/Satellite photos of heavy rain')
test_dl = learn.dls.test_dl(fnames_heavy_rain, with_labels=True)



In [ ]:
from torchvision.transforms.functional import to_tensor
from fastai.vision.all import PILImage, Resize, ResizeMethod, PadMode  # 導入 fastai 模組
fn = test_dl.items[1] #第2張影像
x_dec = PILImage.create(fn);

#Resize : 224填充黑邊
rsz = Resize(224, method=ResizeMethod.Pad, pad_mode=PadMode.Zeros)
x_dec = rsz(x_dec)
x = to_tensor(x_dec)
x.unsqueeze_(0)
x.shape,type(x)


##繪製最後一層的feature map 的Grad-CAM

In [ ]:
cls = 1
with HookBwd(learn.model[0]) as hookg:
  with Hook(learn.model[0]) as hook:
    #output = learn.model.eval()(x,cuda())
    output = learn.model.eval()(x.cpu())
    act = hook.stored
  output[0,cls].backward()
  grad = hookg.stored



In [ ]:
w = grad[0].mean(dim=[1,2], keepdim=True)
cam_map = (w * act[0]).sum(0)

##Demo

In [ ]:
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
           interpolation='bilinear', cmap='magma')
